In [ ]:
!pip install numpy Cython
!pip install pymrmr

     |████████████████████████████████| 71kB 3.4MB/s 
  Created wheel for pymrmr: filename=pymrmr-0.1.11-cp37-cp37m-linux_x86_64.whl size=276575 sha256=99615f4a63666ea50cd4e729b40f3d401baf5dcabb5c44b0a2b0b40686e1112a
  Stored in directory: /root/.cache/pip/wheels/9b/e2/c2/4b232fc0b8128263b518808866f5797070fe6750203ad1027a
Successfully built pymrmr


In [ ]:
import scipy.io as sio
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.pipeline import make_pipeline
import tabulate
from sklearn.model_selection import cross_val_score


#Pltter
import matplotlib.pyplot as plt
import plotly.express as px
from matplotlib.colors import ListedColormap
import plotly.graph_objs as go
import plotly.offline as py

# Classifiers
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier

# Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Selector
import pymrmr

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Read data from mat file
EEG_X = sio.loadmat('/content/drive/My Drive/Colab Notebooks/EEG_X.mat')['X'][0]
EEG_Y = sio.loadmat('/content/drive/My Drive/Colab Notebooks/EEG_Y.mat')['Y'][0]

print("EEG_X shape: ",EEG_X.shape, "   EEG_Y shape: ", EEG_Y.shape)
print("EEG_X[0] shape: ",EEG_X[0].shape, "   EEG_Y[0] shape: ", EEG_Y[0].shape)


EEG_X shape:  (15,)    EEG_Y shape:  (15,)
EEG_X[0] shape:  (3394, 310)    EEG_Y[0] shape:  (3394, 1)


In [ ]:
# To save time, I will create the list manually.
def select_mRMR_feat(X,y,num_feat = 28, ver ="MIQ"):
  if ver == "MIQ" and num_feat == 28:
    return [271,84,226,262,32,279,274,270,209,264,280,208,217,200,62,304,263,288,122,201,39,281,85,218,23,290,251,272]
  else:
    new_df = pd.DataFrame(data = y.to_numpy(),columns=["target"])
    string_names = []
    for column in X:
      new_df[column]= X[column]
    for name in new_df.columns:
      string_names.append(str(name))
    new_df.columns = string_names
    selected_features = pymrmr.mRMR(new_df, ver,num_feat)
    return [int(x) for x in selected_features]

def extract_features(df,selected_features):
  new_df = pd.DataFrame()
  for cols in df:
    if cols in selected_features:
      new_df[cols] = df[cols]
  return new_df

def feat_dim_shrink(X,y):
  lda = LinearDiscriminantAnalysis(n_components=2)

  return lda.fit_transform(X,y)

In [ ]:
# Store all samples in one dataframe
def gck_preprocess(EEG_X,EEG_Y,num_feat=28,mRMR_ver="MIQ",SJTU_DATA = True):
  if SJTU_DATA == True:
    ALL_X = EEG_X[0]
    ALL_Y = EEG_Y[0]
    for idx in range(1,len(EEG_X)):
      ALL_X = np.vstack((ALL_X,EEG_X[idx]))
      ALL_Y = np.vstack((ALL_Y,EEG_Y[idx]))

    df_ALL = pd.DataFrame(data=ALL_X)
    df_ALL['result'] = ALL_Y

    X = df_ALL.drop(['result'],axis=1)
    y = df_ALL['result']

    selected_features = select_mRMR_feat(X,y,num_feat,mRMR_ver)
    extracted_X = extract_features(df_ALL,selected_features)
    shrinked_X = feat_dim_shrink(extracted_X,ALL_Y)

    # NEW_ALL_X = shrinked_X[0]
    # for idx in range(1,len(shrinked_X)):
    #   NEW_ALL_X = np.vstack((NEW_ALL_X,shrinked_X[idx]))

    # df_X = pd.DataFrame(data=NEW_ALL_X,columns=["LD1",'LD2'])
    # df_y = pd.DataFrame(data=ALL_Y,columns=['class'])
    # return df_X.to_numpy(),df_y.to_numpy().flatten()
    return shrinked_X,ALL_Y
  else:
    print("Still in development!")
    return [],[]

X,y = gck_preprocess(EEG_X,EEG_Y)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



In [ ]:
class GKC_MODEL:
  def __init__(self):
    self.scaler = StandardScaler()
    self.selected_features = []
    self.scores = []
    self.clf = SGDClassifier(max_iter= 10000)

  def train(self,X,y):
    self.clf.fit(X,y)
    
    print("Training Complete!")

  def predict(self,X):
    return self.clf.predict(X)

  def score(self,X,y):
    return self.clf.score(X,y) * 100

  def cross_val_score(self,X,y,cv = 15):
    score = cross_val_score(self.clf, X, y, cv=cv)
    print(score,np.mean(score))

  def get_clf(self):
    return self.clf
model = GKC_MODEL()


In [ ]:
#TEST SKLEARN TEST TRAIN SPLIT aND CROSS VALDIATION
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.80, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
model.train(X_train,y_train)

Training Complete!


In [ ]:
model.cross_val_score(X_test,y_test,15)

[0.9937408  0.99410898 0.99447717 0.99558011 0.99484346 0.99373849
 0.99705341 0.99447514 0.99373849 0.99558011 0.99742173 0.99631676
 0.99447514 0.99447514 0.99594843] 0.9950648906825932


In [ ]:
h = .02  # step size in the mesh
X = StandardScaler().fit_transform(X)

# Implement 3 Logistic Regression models with varying values of C



# Define our usual decision surface bounding plots
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1

xx, yy = np.meshgrid(np.arange(x_min, x_max, h)
                     , np.arange(y_min, y_max, h))
y_ = np.arange(y_min, y_max, h)

Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

trace1 = go.Heatmap(x=xx[0], y=y_, z=Z,
                  colorscale='Viridis',
                  showscale=True)

trace2 = go.Scatter(x=X[:, 0], y=X[:, 1], 
                    mode='markers',
                    showlegend=True,
                    marker=dict(size=2,
                                color=y, 
                                colorscale='rdylgn',
                                # line=dict(color='red', width=1)
                                )
                    )
layout= go.Layout(
    autosize= False,
    title= 'GCK Decision Surface',
    showlegend= True)
    
data = [trace1, trace2]
fig = go.Figure(data=data, layout=layout)       
py.iplot(fig)

Output hidden; open in https://colab.research.google.com to view.